 # Buy Box Seller Name via eStore AP

In [2]:
import requests
import pandas as pd


TOKEN = "1476ff45b88bbd6f76074a42c4daf7d8"
COOKIE = "PHPSESSID=b942169d187266534a5484f8effff0d5"
UK_PROJECT_ID = "1181"
UK_PRODUCT_GROUP_ID = "3357"
BB_SELLER_TEMPLATE_ID = "1171"
UK_AMAZON_URL = "http://amazon.co.uk"
DATA_KEY = "data"
PRODUCTS_KEY = "products"
SEPERATOR = "_"
ORIENT_RECORDS = "records"
RAW_VALUE_KEY ="raw_value"
ORIENT_INDEX = "index"
ESTORE_PRODUCT_ID = "estore_product_id"
REGION_DICTIONARY = {
        'de': {'project_id': '3011', 'product_group_id': '4281', 'website_url': 'http://amazon.de'},
        'uk': {'project_id': '1181', 'product_group_id': '3357', 'website_url': 'http://amazon.co.uk'},
        'fr': {'project_id': '3013', 'product_group_id': '4077', 'website_url': 'http://amazon.fr'},
        'es': {'project_id': '3015', 'product_group_id': '4076', 'website_url': 'http://amazon.es'},
        'it': {'project_id': '3017', 'product_group_id': '4274', 'website_url': 'http://amazon.it'},
        'nl': {'project_id': '3790', 'product_group_id': '5061', 'website_url': 'amazon.nl'},
        'se': {'project_id': '3728', 'product_group_id': '4713', 'website_url': 'amazon.se'}
    }


def make_date_start_and_end_list(startdate, enddate):
    """
    Function creates a list holding dates in YYYY-MM-DD format based on specified start and end
    :param startdate: YYYY-MM-DD
    :param enddate: YYYY-MM-DD
    :return: list
    """
    pd_datetime = pd.date_range(startdate, enddate)
    pd_date = [date.date() for date in pd_datetime]
    str_date_list = [str(date) for date in pd_date]
    return str_date_list


def get_estore_api_response(date, token="1476ff45b88bbd6f76074a42c4daf7d8", project_id="1181", product_group_id="3357",  template_id="1171", store_url="http://amazon.co.uk"):
    """
    Function creates a GET call to the eStore API with the specified parameters. The fucntion returns the JSON response.
    :param date: YYYY-MM-DD
    :param token: eStore API Token
    :param project_id: Project ID - eStore Country
    :param product_group_id: Product Group ID - eStore Assortment
    :param template_id: Template ID - eStore Template
    :param store_url: URL of store
    :return: JSON
    """
    url = f"https://www.estorecheck.com/api/oneview?token={token}&project_id={project_id}&product_group_id={product_group_id}&date_start={date}&date_end={date}&template_id={template_id}&store_url={store_url}"
    headers = {'Cookie': COOKIE}
    response = requests.request("GET", url, headers=headers)
    return response.json()


def get_product_info_dataframe(json_normalized, column):
    """
    Function creates a dictionary for each product (column) and returns a dataframe where the
    keys are the headers and the values are the cells.
    param token (pandas dataframe): dataframe of filtered eStore data
    param column (string): name of column to loop through
    returns (pandas dataframe): dataframe containing information on product
    """
    json_normalized_dictionary = json_normalized.to_dict(orient=ORIENT_RECORDS)[0][column][0]
    product_info_dictionary = dict()
    for key in json_normalized_dictionary.keys():
        product_info_dictionary[key] = json_normalized_dictionary[key][RAW_VALUE_KEY]
        product_id = column.replace("_" + UK_AMAZON_URL, "")
        product_info_dictionary[ESTORE_PRODUCT_ID] = product_id
    product_info_dataframe = pd.DataFrame.from_dict(product_info_dictionary, orient=ORIENT_INDEX).T
    return product_info_dataframe


def get_json_to_dataframe(json):
    """
    Function takes the input from an eStore API get call and returns a dataframe with appropriate
    headers and values
    param token (pandas dataframe): eStore JSON data
    returns (pandas dataframe): dataframe with columns for product features and rows of unique products
    """
    json_normalized = pd.json_normalize(json[DATA_KEY][PRODUCTS_KEY], sep=SEPERATOR)
    dataframe = pd.DataFrame()
    for column in json_normalized.columns:
        product_info_dataframe = get_product_info_dataframe(json_normalized, column)
        dataframe = dataframe.append(product_info_dataframe)
    dataframe.reset_index(inplace=True, drop=True)
    return dataframe


def pipeline_single_date_estore_api_response_df(date, token="1476ff45b88bbd6f76074a42c4daf7d8", project_id="1181", product_group_id="3357",  template_id="1171", store_url="http://amazon.co.uk"):
    """
    Function pipelines the process required to call the eStore API and form a readable DataFrame from its JSON response for a single date
    :param date: YYYY-MM-DD
    :param token: eStore API Token
    :param project_id: Project ID - eStore Country
    :param product_group_id: Product Group ID - eStore Assortment
    :param template_id: Template ID - eStore Template
    :param store_url: URL of store
    :return: Dataframe
    """
    response_json = get_estore_api_response(date, token=token, project_id=project_id, product_group_id=product_group_id,  template_id=template_id, store_url=store_url)
    response_dataframe = get_json_to_dataframe(response_json)
    response_dataframe["date"] = date
    response_dataframe["country"] = store_url.split(".")[-1]
    return response_dataframe


def make_pipeline_multiple_date_estore_api_response_df(startdate, enddate, token="1476ff45b88bbd6f76074a42c4daf7d8", project_id="1181", product_group_id="3357",  template_id="1171", store_url="http://amazon.co.uk"):
    """
    Function pipelines the process required to call the eStore API and form a readable DataFrame from its JSON response for a date range
    :param startdate: YYYY-MM-DD
    :param enddate: YYYY-MM-DD
    :param token: eStore API Token
    :param project_id: Project ID - eStore Country
    :param product_group_id: Product Group ID - eStore Assortment
    :param template_id: Template ID - eStore Template
    :param store_url: URL of store
    :return: Dataframe
    """
    str_date_list = make_date_start_and_end_list(startdate, enddate)
    estore_response_df = pd.DataFrame()
    for date in str_date_list:
        print(date)
        date_response_dataframe = pipeline_single_date_estore_api_response_df(date, token=token, project_id=project_id, product_group_id=product_group_id,  template_id=template_id, store_url=store_url)
        estore_response_df = estore_response_df.append(date_response_dataframe)
    return estore_response_df


# Create DF for UK - July through Sept
# buybux_seller_df = make_pipeline_multiple_date_estore_api_response_df("2022-07-01", "2022-09-30", token=TOKEN, project_id=UK_PROJECT_ID, product_group_id=UK_PRODUCT_GROUP_ID,  template_id=BB_SELLER_TEMPLATE_ID, store_url=UK_AMAZON_URL)
# buybux_seller_df


# Create DF for all markets - July through Sept
all_market_bb_seller_name_df = pd.DataFrame()
for region_dict in list(REGION_DICTIONARY.values()):
    print(region_dict["website_url"])
    buybux_seller_df = make_pipeline_multiple_date_estore_api_response_df("2022-07-01", "2022-07-2", token=TOKEN, project_id=region_dict["project_id"], product_group_id=region_dict["product_group_id"],  template_id=BB_SELLER_TEMPLATE_ID, store_url=region_dict["website_url"])
    all_market_bb_seller_name_df = all_market_bb_seller_name_df.append(buybux_seller_df)
all_market_bb_seller_name_df

http://amazon.de
2022-07-01


KeyboardInterrupt: 

### Single Day Data

In [107]:
response_dataframe = pipeline_single_date_estore_api_response_df("2022-09-20")
response_dataframe

TypeError: 'dict' object is not callable

### Date Range Data

In [105]:
buybux_seller_df = make_pipeline_multiple_date_estore_api_response_df("2022-07-01", "2022-09-30", token="1476ff45b88bbd6f76074a42c4daf7d8", project_id="1181", product_group_id="3357",  template_id="1171", store_url="http://amazon.co.uk")

2022-07-01


TypeError: 'dict' object is not callable

### All Markets Date Range